In [1]:
import pandas as pd

In [2]:
features = pd.read_csv("datasets/liss/LISS_example_input_data.csv", encoding = 'cp1252', low_memory = False)

In [4]:
outcome = pd.read_csv("datasets/liss/LISS_example_groundtruth_data.csv", encoding = 'cp1252', low_memory = False)

In [5]:
#features.shape

In [6]:
#outcome.shape

In [7]:
#outcome.tail()

In [8]:
#features.tail()

In [9]:
#features.describe()

In [10]:
#outcome.describe()

In [11]:
#outcome["new_child"].value_counts()

In [12]:
#outcome["new_child"].isna().sum()

In [67]:
feature_selection = ["nomem_encr", "brutocat2019", "sted2019", "woonvorm2019", "burgstat2019", "aantalki2019"]
features_final = features[feature_selection]
features_final

,nomem_encr,brutocat2019,sted2019,woonvorm2019,burgstat2019,aantalki2019
0,800000.0,NaN,NaN,NaN,NaN,NaN
1,800018.0,NaN,NaN,NaN,NaN,NaN
2,800021.0,NaN,NaN,NaN,NaN,NaN
3,800033.0,NaN,NaN,NaN,NaN,NaN
4,800042.0,EUR 1001 to EUR 1500,Extremely urban,"(Un)married co-habitation, with child(ren)",Married,Two children
...,...,...,...,...,...,...
9454,899917.0,NaN,NaN,NaN,NaN,NaN
9455,899935.0,NaN,NaN,NaN,NaN,NaN
9456,899942.0,NaN,NaN,NaN,NaN,NaN
9457,899957.0,NaN,Not urban,"Single, with child(ren)",Never been married,Two children


In [82]:
merged = pd.merge(features_final, outcome, on = "nomem_encr")

In [83]:
merged_clean = merged.dropna()

In [84]:
features_clean = merged_clean.drop(columns = ["new_child", "nomem_encr"])
outcome_clean = merged_clean["new_child"]

In [85]:
features_clean_cat = features_clean.select_dtypes(include = object)
features_clean_num = features_clean.select_dtypes(exclude = object)

In [72]:
### now do some ML

In [112]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

In [74]:
#encoder = OneHotEncoder(handle_unknown = "ignore")
#scaler = StandardScaler()

In [79]:
#preprocessor = ColumnTransformer(
#    [
#        ("one-hot-encoder", encoder, features_clean_cat),
#        ("standard_scaler", scaler, features_clean_num)
#    ]
#)

In [76]:
#model = make_pipeline(preprocessor, LogisticRegression(max_iter = 500))

In [77]:
#cross_validate(model, features_clean, outcome_clean, cv = 5)

In [66]:
#model

In [123]:
model = make_pipeline(OneHotEncoder(handle_unknown = "ignore"), LogisticRegression(max_iter = 500))

In [113]:
model = make_pipeline(OneHotEncoder(handle_unknown = "ignore"), RandomForestClassifier(n_estimators = 1000))

In [124]:
model

Pipeline(steps=[('onehotencoder', OneHotEncoder(handle_unknown='ignore')),
                ('logisticregression', LogisticRegression(max_iter=500))])

In [125]:
cv_result = cross_validate(model, features_clean, outcome_clean, cv = 5)

In [126]:
cv_result

{'fit_time': array([0.00884438, 0.01373839, 0.01430249, 0.00797844, 0.0089345 ]),
 'score_time': array([0.        , 0.        , 0.00199246, 0.00203514, 0.0019958 ]),
 'test_score': array([0.84210526, 0.84210526, 0.80701754, 0.8245614 , 0.85964912])}

In [127]:
cv_result["test_score"].mean(), cv_result["test_score"].std()

(0.8350877192982455, 0.01789129653892206)